Finetune a Pretrained Hugginface model

In [1]:
#Import Libraries
import os

#Data manipulation
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split

#finetuning
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

#Evaluation
from datasets import load_metric



Activate weight and Biases

In [2]:
os.environ["WANDB_DISABLED"] = "false"

Load The Datasets

In [3]:
#load data
train_df=pd.read_csv('dataset\Train.csv')

#check first 5

train_df.head()


,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


AS Noticed in EDA we will drop the rows with missing values

In [4]:
#Drop null rows
train_df.dropna(axis=0,inplace=True)

#check for the presence of null values again
train_df.isna().sum()


tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

Split the Train the dataset into training(for learning ) and evaluation(For computing metrics to capture overfitting) sets

In [5]:
train , eval = train_test_split(train_df,train_size=0.8,stratify=train_df['label'])

check train data

In [6]:
train.head()

,tweet_id,safe_text,label,agreement
1629,8CJWKNXQ,I was gonna get the vaccine for cooties but I ...,-1.0,0.333333
6994,2ZKZKPAV,"""<user> Children can avoid getting forced vacc...",-1.0,1.000000
2050,REHN9VZ3,I want to go to Disneyland but I also don't wa...,0.0,1.000000
135,C9QSY5LE,Really? MMR Shots?,-1.0,1.000000
1641,CQDD6QLM,"New <user> ""Hey Love"" #MMR #ManyMenRecords #Yo...",0.0,1.000000


Check eval

In [7]:
eval.head()

,tweet_id,safe_text,label,agreement
9958,KS6S2LL6,Another Study Finds No Link Between MMR Vaccin...,1.0,1.000000
7361,9D7K4021,I bet our culture and society do more to induc...,1.0,0.666667
742,17D6V4J7,<user> yes &amp; we will Never give up on #Jus...,-1.0,0.666667
5473,RNP8LUNQ,<user> simple about kids not getting vaccinate...,1.0,1.000000
4549,78BIFAOH,Finally!! Someone explains it in a way everyon...,-1.0,0.666667


Check the shapes of both dataset

In [8]:
print(f'Shape of training set: {train.shape}\nShape of evaluation set: {eval.shape}')

Shape of training set: (7999, 4)
Shape of evaluation set: (2000, 4)


Save the splitted data to be loaded as huggingface dataset

In [9]:
train.to_csv('dataset\Train_subset.csv',index=False)
train.to_csv('dataset\eval_subset.csv',index=False)

In [10]:
datasets=load_dataset('csv',data_files={'train':'dataset\Train_subset.csv','eval':'dataset\eval_subset.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/sylvester/.cache/huggingface/datasets/csv/default-acda7b4d4e5876e0/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Create a function to tokenize the data
create a function to transform the labels or call lable encoder


In [11]:
#Create a tokenizer instance
tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')

In [12]:
#function to tokenize text 
def tokenize_data(data):
    return tokenizer(data['safe_text'],padding='max_length')


#function to transform labels
def transfom_label(data):

    #extract label
    label=data['label']
    num=0
    #create contitions
    if label==-1: #Negative
        num=0
    elif label==0: #Neutral
        num=1
    else:
        num=2      #Positive
    return {'labels':num}



Use .map to apply function to dataset, .map speeds up data processing

In [13]:
#The following columns will be removed after tokenization 
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']

#Tokenize the text data
datasets=datasets.map(tokenize_data,batched=True)

#transform the labels
datasets=datasets.map(transfom_label,remove_columns=remove_columns)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

# Training a Bert model


Extract the train and eval datasets from datasets

In [14]:
#extract train datasets
train_dataset=datasets['eval'].shuffle(seed=0)
#extract eval datasets
eval_dataset=datasets['eval'].shuffle(seed=0)

Loading cached shuffled indices for dataset at C:\Users\sylvester\.cache\huggingface\datasets\csv\default-acda7b4d4e5876e0\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-a940471467d7e511.arrow


In [15]:
#set the training arguments
trainargs=TrainingArguments('training\.test',num_train_epochs=1000,evaluation_strategy="epoch",save_strategy='epoch',load_best_model_at_end=3)

#Create an instance of the bert model
model=AutoModelForSequenceClassification.from_pretrained('bert-base-cased',num_labels=3)

#create a trainer instance
trainer=Trainer(model=model,train_dataset=train_dataset,eval_dataset=eval_dataset,tokenizer=tokenizer)

c:\Users\sylvester\Desktop\Azubi data analyst\career accelerator_clones\LP2\NLP-Sentiment Analysis\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sylvester\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-cased were not u